In [1]:
import pandas as pd
import numpy as np

In [2]:
df_data_villes = pd.read_csv("villes_cp.csv", 
                             delimiter=";",
                             encoding="latin1")

In [3]:
#on clean la colonne nom de la commune
df_data_villes['Nom_de_la_commune'] = (
    df_data_villes['Nom_de_la_commune']
    .str.lower()                         # mettre en minuscule
    .str.replace(" ", "-", regex=False)  # remplacer les espaces par des "-"
)
df_data_villes['Nom_de_la_commune'] = df_data_villes['Nom_de_la_commune'].str.replace(
    r'^st', 'saint', case=False, regex=True # remplacer les st par saint
)


#permet d'avoir 5 caractères dans tous les cas donc 4400 --> 04400
df_data_villes['Code_postal'] = df_data_villes['Code_postal'].astype(str).str.zfill(5)

#Création de la colonne url
df_data_villes['url'] = (
    "https://www.meilleursagents.com/prix-immobilier/" +
    df_data_villes['Nom_de_la_commune'] +
    "-" +
    df_data_villes['Code_postal'].astype(str) + 
    "/"
)

#je supprime les lignes avec la même ville et donc le même url
df_data_villes_clean = df_data_villes.drop_duplicates(subset=['Nom_de_la_commune'])

#je supprime les colonnes inutiles
df_data_villes_clean = df_data_villes_clean.drop(columns=['Ligne_5'])
df_data_villes_clean = df_data_villes_clean.drop(columns=['Libellé_d_acheminement'])

In [ ]:
#Test

In [4]:
#Ici on enregistre le nouveau df en entier
df_data_villes_clean.to_csv('villes_cp_clean.csv')

In [5]:
#Ici on divise et enregistre le nouveau df en 15
number_of_chunks = 15
for i, chunk in enumerate(np.array_split(df_data_villes_clean, number_of_chunks)):
    chunk.to_csv(f'output_{i+1}.csv')

/Users/amauryguilbaud/Library/Python/3.9/lib/python/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
